## 05_Modeling.ipynb
###  **🎯 Objective:**

The objective of this notebook is to define, compile, and train a Convolutional Neural Network (CNN) model to detect powdery mildew in cherry leaf images. This phase follows the preprocessing stage and forms the core of the modeling step in the CRISP-DM methodology.



## Model Architecture
We define a deep learning model using TensorFlow Keras. The model consists of:
 - Convolutional layers for feature extraction
 - MaxPooling layers to reduce spatial dimensions
 - GlobalAveragePooling to reduce the feature map
 - Dense layers for decision making
 - Dropout layer for regularization
 - Sigmoid activation for binary classification

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

### Compile the Model
We compile the model using:
 - Loss function: binary_crossentropy (for binary classification)
 - Optimizer: adam
 - Evaluation metric: accuracy

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

 ### Load Train and Validation Data

### Load Preprocessed Data
Using ImageDataGenerator, we load:
 - Augmented training images
 - Rescaled validation images

This step prepares the model to be trained on realistic and varied data, improving generalization.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_shape = (256, 256)
batch_size = 32

train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

test_aug = ImageDataGenerator(rescale=1./255)

train_data = train_aug.flow_from_directory(
    "../inputs/split-leaves/train",
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='binary'
)

val_data = test_aug.flow_from_directory(
    "../inputs/split-leaves/validation",
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


### Train the Model and Save Artifacts

### Training the Model
We train the model using model.fit(...) with:
 - EarlyStopping to prevent overfitting
 - ModelCheckpoint to save the best-performing model based on validation loss
 - We run the model for up to 20 epochs, monitoring both training and validation accuracy/loss.

In [ ]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("../outputs/mildew_model.h5", save_best_only=True)
]

history = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=callbacks
)

### Save Model and History
We save the trained model to outputs/mildew_model.h5 and store the training history in:
 - training_history.pkl (binary format)
 - history.json (readable format)

This allows us to later visualize training progress without re-training the model.

In [ ]:
# Save training history
import json, pickle

with open("../outputs/training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)

with open("../outputs/history.json", "w") as f:
    json.dump(history.history, f)


#### To avoid retraining every time, comment out model.fit(...) and just use this:

In [ ]:
# with open("../outputs/training_history.pkl", "rb") as f:
#    history_data = pickle.load(f)